# Notebook 07: Autoscaling Simulation

## Mục Tiêu
- Mô phỏng autoscaling với các strategies khác nhau
- So sánh Reactive vs Predictive scaling
- Phân tích chi phí và hiệu suất
- Tối ưu hóa scaling policies

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings

warnings.filterwarnings('ignore')

# Thêm src vào path
sys.path.insert(0, os.path.abspath('..'))

from src.data.preprocessor import load_timeseries, split_train_test
from src.autoscaling.policy import ServerConfig, ScalingPolicy, AutoscalingEngine
from src.autoscaling.simulator import AutoscalingSimulator, SimulationResult
from src.autoscaling.cost_analyzer import CostAnalyzer, CostMetrics
from src.models.lightgbm_forecaster import LightGBMForecaster
from src.features.feature_engineering import TimeSeriesFeatureEngineer

# Settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("Libraries loaded successfully!")

## 1. Load Data và Models

In [ ]:
# Load time series
df = load_timeseries('../data/processed/timeseries_15min.parquet')
df_clean = df[df['is_storm_period'] == 0].copy()

# Use test data for simulation
train, test = split_train_test(df_clean, test_start='1995-08-23')

print(f"Simulation period: {test.index.min()} to {test.index.max()}")
print(f"Simulation length: {len(test)} intervals ({len(test)*15/60:.1f} hours)")

In [ ]:
# Load trained model for predictions
try:
    model = LightGBMForecaster.load('../models/lightgbm_15min.pkl')
    print("LightGBM model loaded!")
    
    # Prepare features for predictions
    fe = TimeSeriesFeatureEngineer(df_clean)
    df_features = fe.create_all_features(target_col='request_count', granularity='15min')
    feature_cols = fe.get_feature_columns(df_features)
    X, y = fe.prepare_supervised(df_features, 'request_count', feature_cols, forecast_horizon=1)
    
    test_mask = X.index >= '1995-08-23'
    X_test = X[test_mask]
    
    # Generate predictions
    predictions = model.predict(X_test)
    predicted_demand = pd.Series(predictions, index=X_test.index)
    print(f"Predictions generated: {len(predicted_demand)}")
    
except Exception as e:
    print(f"Could not load model: {e}")
    print("Using simple moving average for predictions...")
    predicted_demand = test['request_count'].rolling(4).mean().shift(1).fillna(method='bfill')

## 2. Configuration

In [ ]:
# Server Configuration
server_config = ServerConfig(
    max_requests_per_min=1000,      # 1000 requests/min per server
    max_bytes_per_min=50_000_000,   # 50MB/min per server
    min_servers=1,
    max_servers=50,
    cost_per_server_hour=0.10       # $0.10/hour per server
)

print("Server Configuration:")
print(f"  Max Requests/min: {server_config.max_requests_per_min:,}")
print(f"  Min Servers: {server_config.min_servers}")
print(f"  Max Servers: {server_config.max_servers}")
print(f"  Cost: ${server_config.cost_per_server_hour}/hour")

In [ ]:
# Scaling Policy
scaling_policy = ScalingPolicy(
    scale_out_threshold=0.8,      # Scale out at 80% utilization
    scale_in_threshold=0.3,       # Scale in at 30% utilization  
    cooldown_period=5,            # 5 intervals (75 min) cooldown
    consecutive_breaches=3,       # Need 3 consecutive breaches
    scale_out_increment=2,        # Add 2 servers
    scale_in_decrement=1,         # Remove 1 server
    prediction_horizon=4,         # Look 1 hour ahead (4 * 15min)
    predictive_buffer=0.2         # 20% safety buffer
)

print("\nScaling Policy:")
print(f"  Scale Out Threshold: {scaling_policy.scale_out_threshold*100}%")
print(f"  Scale In Threshold: {scaling_policy.scale_in_threshold*100}%")
print(f"  Cooldown: {scaling_policy.cooldown_period} intervals")
print(f"  Consecutive Breaches: {scaling_policy.consecutive_breaches}")
print(f"  Predictive Buffer: {scaling_policy.predictive_buffer*100}%")

## 3. Create Simulator

In [ ]:
# Initialize Simulator
simulator = AutoscalingSimulator(
    server_config=server_config,
    scaling_policy=scaling_policy
)

In [ ]:
# Prepare actual demand series
actual_demand = test['request_count'].copy()
print(f"Demand statistics:")
print(f"  Mean: {actual_demand.mean():.0f} requests/interval")
print(f"  Max: {actual_demand.max():.0f} requests/interval")
print(f"  Min: {actual_demand.min():.0f} requests/interval")

## 4. Run Simulations

### 4.1 Reactive Scaling (Based on Current Load)

In [ ]:
# Reactive scaling simulation
print("Running Reactive Scaling Simulation...")
reactive_result = simulator.run_simulation(
    actual_demand=actual_demand,
    predicted_demand=None,  # No predictions = reactive only
    initial_servers=5,
    freq_minutes=15
)

print(f"\nReactive Scaling Results:")
print(f"  Total Cost: ${reactive_result.total_cost:.2f}")
print(f"  Avg Servers: {reactive_result.average_servers:.1f}")
print(f"  Scaling Events: {reactive_result.scaling_events}")
print(f"  SLA Violations: {reactive_result.sla_violations}")
print(f"  Dropped Requests: {reactive_result.dropped_requests:,}")

### 4.2 Predictive Scaling (Based on Forecasts)

In [ ]:
# Predictive scaling simulation
print("Running Predictive Scaling Simulation...")

# Align predictions with actual demand
pred_aligned = predicted_demand.reindex(actual_demand.index).fillna(method='ffill')

predictive_result = simulator.run_simulation(
    actual_demand=actual_demand,
    predicted_demand=pred_aligned,
    initial_servers=5,
    freq_minutes=15
)

print(f"\nPredictive Scaling Results:")
print(f"  Total Cost: ${predictive_result.total_cost:.2f}")
print(f"  Avg Servers: {predictive_result.average_servers:.1f}")
print(f"  Scaling Events: {predictive_result.scaling_events}")
print(f"  SLA Violations: {predictive_result.sla_violations}")
print(f"  Dropped Requests: {predictive_result.dropped_requests:,}")

## 5. Strategy Comparison

In [ ]:
# Compare all strategies
strategies = {
    'Reactive': reactive_result,
    'Predictive': predictive_result
}

comparison_df = simulator.compare_strategies(
    actual_demand=actual_demand,
    predicted_demand=pred_aligned,
    initial_servers=5,
    freq_minutes=15
)

print("\nStrategy Comparison:")
print(comparison_df.to_string(index=False))

In [ ]:
# Visual comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Cost comparison
costs = [reactive_result.total_cost, predictive_result.total_cost]
axes[0, 0].bar(['Reactive', 'Predictive'], costs, color=['coral', 'steelblue'])
axes[0, 0].set_title('Total Cost ($)')
axes[0, 0].set_ylabel('Cost ($)')
for i, v in enumerate(costs):
    axes[0, 0].text(i, v + 0.5, f'${v:.2f}', ha='center')

# Average servers
avg_servers = [reactive_result.average_servers, predictive_result.average_servers]
axes[0, 1].bar(['Reactive', 'Predictive'], avg_servers, color=['coral', 'steelblue'])
axes[0, 1].set_title('Average Servers')
axes[0, 1].set_ylabel('Servers')
for i, v in enumerate(avg_servers):
    axes[0, 1].text(i, v + 0.1, f'{v:.1f}', ha='center')

# Scaling events
events = [reactive_result.scaling_events, predictive_result.scaling_events]
axes[1, 0].bar(['Reactive', 'Predictive'], events, color=['coral', 'steelblue'])
axes[1, 0].set_title('Scaling Events')
axes[1, 0].set_ylabel('Count')
for i, v in enumerate(events):
    axes[1, 0].text(i, v + 0.5, str(v), ha='center')

# SLA violations
violations = [reactive_result.sla_violations, predictive_result.sla_violations]
axes[1, 1].bar(['Reactive', 'Predictive'], violations, color=['coral', 'steelblue'])
axes[1, 1].set_title('SLA Violations')
axes[1, 1].set_ylabel('Count')
for i, v in enumerate(violations):
    axes[1, 1].text(i, v + 0.5, str(v), ha='center')

plt.suptitle('Reactive vs Predictive Scaling', fontsize=14)
plt.tight_layout()
plt.savefig('../reports/figures/autoscaling_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Timeline Visualization

In [ ]:
# Plot server allocation over time
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Demand
axes[0].plot(actual_demand.index, actual_demand.values, color='black', label='Actual Demand')
axes[0].plot(pred_aligned.index, pred_aligned.values, color='green', alpha=0.5, 
             linestyle='--', label='Predicted Demand')
axes[0].set_ylabel('Requests')
axes[0].set_title('Traffic Demand')
axes[0].legend()

# Reactive servers
reactive_servers = reactive_result.timeline['servers']
axes[1].plot(actual_demand.index[:len(reactive_servers)], reactive_servers, 
             color='coral', label='Reactive', linewidth=2)
axes[1].set_ylabel('Servers')
axes[1].set_title('Server Allocation - Reactive')
axes[1].legend()

# Predictive servers
predictive_servers = predictive_result.timeline['servers']
axes[2].plot(actual_demand.index[:len(predictive_servers)], predictive_servers, 
             color='steelblue', label='Predictive', linewidth=2)
axes[2].set_ylabel('Servers')
axes[2].set_title('Server Allocation - Predictive')
axes[2].set_xlabel('Timestamp')
axes[2].legend()

plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../reports/figures/autoscaling_timeline.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Utilization comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

reactive_util = reactive_result.timeline['utilization']
predictive_util = predictive_result.timeline['utilization']

axes[0].plot(actual_demand.index[:len(reactive_util)], reactive_util, color='coral')
axes[0].axhline(y=scaling_policy.scale_out_threshold, color='red', linestyle='--', 
                label=f'Scale Out ({scaling_policy.scale_out_threshold*100}%)')
axes[0].axhline(y=scaling_policy.scale_in_threshold, color='green', linestyle='--',
                label=f'Scale In ({scaling_policy.scale_in_threshold*100}%)')
axes[0].fill_between(actual_demand.index[:len(reactive_util)], 0, reactive_util, alpha=0.3, color='coral')
axes[0].set_ylabel('Utilization')
axes[0].set_title('Server Utilization - Reactive')
axes[0].legend(loc='upper right')
axes[0].set_ylim(0, 1.2)

axes[1].plot(actual_demand.index[:len(predictive_util)], predictive_util, color='steelblue')
axes[1].axhline(y=scaling_policy.scale_out_threshold, color='red', linestyle='--')
axes[1].axhline(y=scaling_policy.scale_in_threshold, color='green', linestyle='--')
axes[1].fill_between(actual_demand.index[:len(predictive_util)], 0, predictive_util, alpha=0.3, color='steelblue')
axes[1].set_ylabel('Utilization')
axes[1].set_title('Server Utilization - Predictive')
axes[1].set_xlabel('Timestamp')
axes[1].set_ylim(0, 1.2)

plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../reports/figures/autoscaling_utilization.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Cost Analysis

In [ ]:
# Cost analyzer
cost_analyzer = CostAnalyzer(server_config)

# Generate cost report
reactive_servers_series = pd.Series(
    reactive_result.timeline['servers'], 
    index=actual_demand.index[:len(reactive_result.timeline['servers'])]
)

report = cost_analyzer.generate_cost_report(
    demand_series=actual_demand[:len(reactive_servers_series)],
    server_series=reactive_servers_series,
    freq_minutes=15
)

print(report)

In [ ]:
# Cost comparison with fixed provisioning
demand_aligned = actual_demand[:len(predictive_servers)]

# Calculate costs for different strategies
fixed_peak, fixed_peak_metrics = cost_analyzer.calculate_optimal_fixed_servers(
    demand_aligned, sla_target=99.0, freq_minutes=15
)

print(f"\nOptimal Fixed Servers for 99% SLA: {fixed_peak}")
print(f"Fixed Cost: ${fixed_peak_metrics.total_cost:.2f}")
print(f"\nComparison:")
print(f"  Fixed Provisioning: ${fixed_peak_metrics.total_cost:.2f}")
print(f"  Reactive Scaling: ${reactive_result.total_cost:.2f}")
print(f"  Predictive Scaling: ${predictive_result.total_cost:.2f}")

savings_reactive = fixed_peak_metrics.total_cost - reactive_result.total_cost
savings_predictive = fixed_peak_metrics.total_cost - predictive_result.total_cost

print(f"\nSavings vs Fixed:")
print(f"  Reactive: ${savings_reactive:.2f} ({savings_reactive/fixed_peak_metrics.total_cost*100:.1f}%)")
print(f"  Predictive: ${savings_predictive:.2f} ({savings_predictive/fixed_peak_metrics.total_cost*100:.1f}%)")

## 8. Sensitivity Analysis

In [ ]:
# Test different buffer sizes
print("Running sensitivity analysis on predictive buffer...")

buffer_results = simulator.run_sensitivity_analysis(
    actual_demand=actual_demand,
    predicted_demand=pred_aligned,
    parameter='predictive_buffer',
    values=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    initial_servers=5,
    freq_minutes=15
)

print(buffer_results[['Parameter Value', 'Total Cost', 'Avg Servers', 
                      'SLA Violations', 'Dropped Requests']].to_string(index=False))

In [ ]:
# Visualize sensitivity analysis
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

buffer_values = buffer_results['Parameter Value'].values

# Cost vs Buffer
axes[0].plot(buffer_values, buffer_results['Total Cost'], marker='o', color='steelblue')
axes[0].set_xlabel('Predictive Buffer')
axes[0].set_ylabel('Total Cost ($)')
axes[0].set_title('Cost vs Buffer Size')

# SLA Violations vs Buffer
axes[1].plot(buffer_values, buffer_results['SLA Violations'], marker='o', color='coral')
axes[1].set_xlabel('Predictive Buffer')
axes[1].set_ylabel('SLA Violations')
axes[1].set_title('SLA Violations vs Buffer Size')

# Avg Servers vs Buffer
axes[2].plot(buffer_values, buffer_results['Avg Servers'], marker='o', color='seagreen')
axes[2].set_xlabel('Predictive Buffer')
axes[2].set_ylabel('Average Servers')
axes[2].set_title('Avg Servers vs Buffer Size')

plt.tight_layout()
plt.savefig('../reports/figures/autoscaling_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Summary

In [ ]:
print("="*70)
print("              AUTOSCALING SIMULATION SUMMARY")
print("="*70)
print(f"\nSimulation Period: {test.index.min()} to {test.index.max()}")
print(f"Duration: {len(test)*15/60:.1f} hours ({len(test)} intervals)")
print(f"\nServer Configuration:")
print(f"  Capacity: {server_config.max_requests_per_min} requests/min")
print(f"  Cost: ${server_config.cost_per_server_hour}/hour")
print(f"\n" + "-"*70)
print("STRATEGY COMPARISON:")
print("-"*70)
print(f"{'Strategy':<15} {'Cost ($)':<12} {'Avg Servers':<12} {'SLA Violations':<15} {'Dropped Req':<12}")
print("-"*70)
print(f"{'Fixed Peak':<15} {fixed_peak_metrics.total_cost:<12.2f} {fixed_peak:<12} {fixed_peak_metrics.underprovisioning_events:<15} {fixed_peak_metrics.dropped_requests:<12,}")
print(f"{'Reactive':<15} {reactive_result.total_cost:<12.2f} {reactive_result.average_servers:<12.1f} {reactive_result.sla_violations:<15} {reactive_result.dropped_requests:<12,}")
print(f"{'Predictive':<15} {predictive_result.total_cost:<12.2f} {predictive_result.average_servers:<12.1f} {predictive_result.sla_violations:<15} {predictive_result.dropped_requests:<12,}")
print("-"*70)
print(f"\nKEY FINDINGS:")
print(f"  - Predictive scaling saves ${savings_predictive:.2f} vs fixed provisioning")
print(f"  - Cost reduction: {savings_predictive/fixed_peak_metrics.total_cost*100:.1f}%")
best_buffer = buffer_results.loc[buffer_results['SLA Violations'].idxmin(), 'Parameter Value']
print(f"  - Optimal buffer: {best_buffer*100:.0f}% (minimizes SLA violations)")
print("="*70)